## 4.1 Modélisation de l'héritage avec une table unique

### Implémentation sous SQLAlchemy avec illustration d'une relation 1:n

In [1]:
from sqlalchemy import Table, Column,  ForeignKey, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker,  relationship

# Moteur d'accès à la base sqlalchemy.sqlite
engine = create_engine('sqlite:///sqlalchemy.sqlite')

In [2]:
# Session
DBSession = sessionmaker(bind=engine)
session = DBSession()

<p id="declarations" style="font-size:120%">Déclaration des classes métier :</p>

In [3]:
# Classe mère
Base = declarative_base()
Base.metadata.bind = engine

# Définition des classes 
class Person(Base):
    __tablename__ = 'poly_person'

    id = Column(Integer, primary_key=True)
    first_name = Column(String(64), nullable=False)
    last_name = Column(String(64), nullable=False)
    type = Column(String(20), nullable=False)

    __mapper_args__ = {
        "polymorphic_on": type,
        "polymorphic_identity": "person",
    }
    
    def __str__(self):
        return '{} {} {} {}'.format(self.id,self.type,self.first_name,self.last_name)

class Developer(Person):
    login = Column(String(32))
    pwd = Column(String(32))
    
    office_id = Column(Integer,ForeignKey("poly_office.id"))
    office = relationship("DeveloperOffice", uselist=False, back_populates="developer")

    __mapper_args__ = {
        "polymorphic_identity": "developer",
    }

    def __str__(self):
        return super().__str__() + ' {} {} in office {}'.format(
            self.login,
            self.pwd,
            self.office.room if self.office else None)

class Player(Person):
    pseudo = Column(String(16))
    score = Column(Integer)
    
    __mapper_args__ = {
        "polymorphic_identity": "player",
    }

    def __str__(self):
        return super().__str__() + ' {} {}'.format(self.pseudo,self.score)

class VIP(Player):
    email = Column(String(255))
    preferences = Column(String(255)) 

    __mapper_args__ = {
        "polymorphic_identity": "VIP",
    }

    def __str__(self):
        return super().__str__() + ' {} {}'.format(self.email,self.preferences)

class DeveloperOffice(Base):
    __tablename__ = 'poly_office'
    
    id = Column(Integer, primary_key=True)
    room = Column(String(128),nullable=False)

    developer=relationship("Developer", back_populates="office")
    
    def __str__(self):
        return '{} {}'.format(self.id,self.room)
    
# Création éventuelle des tables dans la base
Base.metadata.create_all(engine)

<p id="usage" style="font-size:120%">Usage d'une relation dans le contexte d'une classe fille</p>

In [4]:
# Création éventuelle d'un developer
devs = session.query(Developer).all()
if len(devs) == 0:
    dev = Developer(first_name="Maud",last_name='Zarella',login="mozarella",pwd='di Bufala')
    session.add(dev)
    session.commit()
else:
    dev = devs[0]

print(dev)

2 developer Maud Zarella mozarella di Bufala in office Deck RC 10


In [5]:
# Création éventuelle d'un bureau
offices = session.query(DeveloperOffice).all()
if len(offices) == 0:
    office = DeveloperOffice(room="Deck RC 10")
    session.add(office)
    session.commit()
else:
    office = offices[0]

print(office)

2 Deck RC 10


In [6]:
# Affectation d'un bureau à un développeur
dev = session.query(Developer).first()
dev.office = office

session.add(office)
session.commit()

print(dev)

2 developer Maud Zarella mozarella di Bufala in office Deck RC 10


In [7]:
# Vérification dans la base
check_dev = session.query(Developer).filter(Developer.id == dev.id).one()
print (check_dev.first_name,check_dev.last_name,check_dev.office.room)

check_office = session.query(DeveloperOffice). \
    join(Developer). \
    filter(Developer.office_id == office.id). \
    one()
    
print(check_office, check_office.developer[0].first_name,check_office.developer[0].last_name)

Maud Zarella Deck RC 10
2 Deck RC 10 Maud Zarella


In [8]:
# liste polymorphique des personnes
people = session.query(Person).all()
for person in people:
    print(person)

1 person Raymond Deubaze
2 developer Maud Zarella mozarella di Bufala in office Deck RC 10


In [9]:
# suppression du bureau
session.delete(check_office)
session.commit()